#**라이브러리 다운**

In [ ]:
!pip install text_hammer
!pip install transformers
!pip install tensorflow
!pip install kaggle --updrade
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

from transformers import AutoTokenizer, TFBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --updrade


In [ ]:
from google.colab import files
files.upload()
# 파일 선택 버튼 클릭 후 kaggle.json 파일 업로드

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tfghji","key":"3992ff8170c8f13c82c7acbb158f5823"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!ls -1ha kaggle.json

kaggle.json


In [ ]:
!kaggle datasets download -d parulpandey/emotion-dataset

100% 715k/715k [00:00<00:00, 838kB/s]
100% 715k/715k [00:00<00:00, 838kB/s]


In [ ]:
# 확인
!ls
# 압축 풀기
!unzip emotion-dataset.zip

emotion-dataset.zip  kaggle.json  sample_data
Archive:  emotion-dataset.zip
  inflating: test.csv                
  inflating: training.csv            
  inflating: validation.csv          


In [ ]:
import pandas as pd
import numpy as np
import text_hammer as th
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer,TFBertModel
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import shutil
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False # if autocompletion doesnot work in kaggle notebook | hit tab

In [ ]:
# importing the dataset
df_train = pd.read_csv('/content/training.csv')
df_test = pd.read_csv('/content/test.csv')
df_val=pd.read_csv('/content/validation.csv')

In [ ]:
df_full = pd.concat([df_train,df_test,df_val], axis = 0)
df_full

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
1995,im having ssa examination tomorrow in the morn...,0
1996,i constantly worry about their fight against n...,1
1997,i feel its important to share this info for th...,1
1998,i truly feel that if you are passionate enough...,1


#**텍스트 사전작업**


In [ ]:
def text_preprocessing(df,col_name):
    column = col_name
    df[column] = df[column].progress_apply(lambda x:str(x).lower())
    df[column] = df[column].progress_apply(lambda x: th.cont_exp(x)) #you're -> you are; i'm -> i am
    df[column] = df[column].progress_apply(lambda x: th.remove_emails(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_html_tags(x))

    df[column] = df[column].progress_apply(lambda x: th.remove_special_chars(x))
    df[column] = df[column].progress_apply(lambda x: th.remove_accented_chars(x))

    return(df)

In [ ]:
df_cleaned = text_preprocessing(df_full,'text')

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
df_cleaned = df_cleaned.copy()
df_cleaned['num_words'] = df_cleaned['text'].apply(lambda x:len(x.split()))
# changing the data type to the category to encode into codes
df_cleaned['label'] = df_cleaned['label'].astype('category')
df_cleaned['label']

0       0
1       0
2       3
3       2
4       3
       ..
1995    0
1996    1
1997    1
1998    1
1999    1
Name: label, Length: 20000, dtype: category
Categories (6, int64): [0, 1, 2, 3, 4, 5]

In [ ]:
df_cleaned['label'].cat.codes

0       0
1       0
2       3
3       2
4       3
       ..
1995    0
1996    1
1997    1
1998    1
1999    1
Length: 20000, dtype: int8

In [ ]:
encoded_dict  = {'anger':0,'fear':1, 'joy':2, 'love':3, 'sadness':4, 'surprise':5}
df_cleaned.num_words.max()

66

In [ ]:
data_train,data_test = train_test_split(df_cleaned, test_size = 0.3, random_state = 42, stratify = df_cleaned['label'])
data_train.shape

(14000, 3)

In [ ]:
data_test.shape

(6000, 3)

In [ ]:
to_categorical(data_train['label'])

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

#**모델 로딩**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# for saving model locally and we can load it later on
tokenizer.save_pretrained('bert-tokenizer')
bert.save_pretrained('bert-model')
shutil.make_archive('bert-tokenizer', 'zip', 'bert-tokenizer')
shutil.make_archive('bert-model','zip','bert-model')

'/content/bert-model.zip'

In [ ]:
tokenizer('I will be kaggle master')

{'input_ids': [101, 146, 1209, 1129, 24181, 25186, 3283, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# Tokenize the input (takes some time)
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=data_train['text'].tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


x_test = tokenizer(
    text=data_test['text'].tolist(),
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test['input_ids']

<tf.Tensor: shape=(6000, 70), dtype=int32, numpy=
array([[ 101,  178, 1243, ...,    0,    0,    0],
       [ 101,  178, 1631, ...,    0,    0,    0],
       [ 101,  178, 2810, ...,    0,    0,    0],
       ...,
       [ 101,  178, 1838, ...,    0,    0,    0],
       [ 101,  178, 1238, ...,    0,    0,    0],
       [ 101,  178, 4534, ...,    0,    0,    0]], dtype=int32)>

In [ ]:
max_len = 70

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

embeddings = bert(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(6,activation = 'sigmoid')(out)

model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True
optimizer = Adam(
    learning_rate=5e-05,  # 이는 huggingface 웹사이트에서 가져온 bert 모델의 학습률입니다.
    epsilon=1e-08,
   # beta_1=0.9,  # 이 값은 beta_1로 설정됩니다.
   # beta_2=0.999,  # beta_2 값도 설정할 수 있습니다.
)


# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = metric)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 70)]                 0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 70)]                 0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, 70, 76                                     

In [ ]:
tf.config.experimental_run_functions_eagerly(True)
tf.config.run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


#**모델 적용과 확인**

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = to_categorical(data_train['label']),
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, to_categorical(data_test['label'])
    ),
  epochs=7,
    batch_size=24
)

Epoch 1/7


  1/584 [..............................] - ETA: 35:25 - loss: 1.6535 - accuracy: 0.2917

  2/584 [..............................] - ETA: 24:33 - loss: 1.5708 - accuracy: 0.3542

  3/584 [..............................] - ETA: 19:09 - loss: 1.5867 - accuracy: 0.3056

  4/584 [..............................] - ETA: 17:17 - loss: 1.6069 - accuracy: 0.2917

...

117/584 [=====>........................] - ETA: 10:46 - loss: 1.5857 - accuracy: 0.3422

118/584 [=====>........................] - ETA: 10:44 - loss: 1.5866 - accuracy: 0.3425

119/584 [=====>........................] - ETA: 10:43 - loss: 1.5860 - accuracy: 0.3428

120/584 [=====>........................] - ETA: 10:41 - loss: 1.5862 - accuracy: 0.3420

121/584 [=====>........................] - ETA: 10:38 - loss: 1.5889 - accuracy: 0.3409

#**예측**

In [ ]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

188/188 [==============================] - 55s 292ms/step


array([0.7208359 , 0.6622844 , 0.46982473, 0.4934486 , 0.28568357,
       0.24297796], dtype=float32)

In [ ]:
y_predicted = np.argmax(predicted_raw, axis = 1)
y_predicted

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
data_test['label']

2950     1
11631    1
8685     1
10305    0
11829    0
        ..
2666     1
6556     3
14591    0
1748     3
4950     4
Name: label, Length: 6000, dtype: category
Categories (6, int64): [0, 1, 2, 3, 4, 5]

In [ ]:
accuracy_score(data_test['label'],y_predicted)

0.28983333333333333

In [ ]:
print(classification_report(data_test['label'], y_predicted))

              precision    recall  f1-score   support

           0       0.29      1.00      0.45      1739
           1       0.00      0.00      0.00      2028
           2       0.00      0.00      0.00       492
           3       0.00      0.00      0.00       813
           4       0.00      0.00      0.00       712
           5       0.00      0.00      0.00       216

    accuracy                           0.29      6000
   macro avg       0.05      0.17      0.07      6000
weighted avg       0.08      0.29      0.13      6000



#**입력 텍스트의 예측**

In [ ]:
texts = input(str('input the text'))

x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding='max_length',
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
validation

input the texttoday i was sad
1/1 [==============================] - 0s 307ms/step


array([[72.083595, 66.22843 , 46.98247 , 49.344864, 28.568357, 24.29779 ]],
      dtype=float32)

In [ ]:
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

anger 72.083595
fear 66.22843
joy 46.98247
love 49.344864
sadness 28.568357
surprise 24.29779
